## VirES - magnetic model benchmark

Assess perfomance of the VirES magnetic model evaluation and other parameters wrt. the plain data rerival. 
Get per sample times.

See also [VirES Python Client](https://github.com/ESA-VirES/VirES-Python-Client)

In [8]:
from viresclient import SwarmRequest

request = SwarmRequest()


SERVER_URL = "https://vires.services"
OWS_URL = f"{SERVER_URL}/ows"
HAPI_URL = f"{SERVER_URL}/hapi"

In [19]:
import requests
from viresclient._wps.time_util import parse_datetime

def get_collection_date_range(collection):
    """ Get collection start and stop dates. """
    info = requests.get(f"{HAPI_URL}/info?dataset={collection}").json()
    return {
        "startDate": parse_datetime(info["startDate"]),
        "stopDate": parse_datetime(info["stopDate"]),
    }
    
print(get_collection_date_range("SW_OPER_MAGA_LR_1B"))


{'startDate': datetime.datetime(2013, 11, 25, 11, 2, 52), 'stopDate': datetime.datetime(2022, 10, 23, 23, 59, 59)}


In [34]:
from datetime import datetime


def get_request_time(collection, start_time, end_time, auxiliaries=None, models=None, filter=None):
    request = SwarmRequest(OWS_URL)
    request.set_collection(collection)
    request.set_products(
        measurements=["B_NEC"],
        auxiliaries=auxiliaries,
        models=models,
    )
    if filter:
        request.add_filter(filter)
    request_start = datetime.now()
    data = request.get_between(
        start_time=start_time,
        end_time=end_time,
        asynchronous=False,
    )
    request_stop = datetime.now()
    xdata = data.as_xarray() 
    size = xdata["Timestamp"].shape[0]
    print(size, list(xdata))
    # time per sample in seconds
    return (request_stop - request_start).total_seconds() / size


In [ ]:
COLLECTION = {
    "A": "SW_OPER_MAGA_LR_1B",
    "B": "SW_OPER_MAGB_LR_1B",
    "C": "SW_OPER_MAGC_LR_1B",
}

SPACECRAFT = "A"


# get time-span of the collection
print(get_collection_date_range(COLLECTION[SPACECRAFT]))


# plain data request
time = get_request_time(
    collection=COLLECTION["A"],
    start_time = "2019-06-01", # both iso-date/time string or datetime.datetime object is accepted
    end_time = "2019-06-02",
)
print(f"{time:e} s per sample")


# plain data request with filter
time = get_request_time(
    collection=COLLECTION["A"],
    start_time = "2019-06-01", # both iso-date/time string or datetime.datetime object is accepted
    end_time = "2019-06-02",
    filter="Flags_B != 255", # strip no-dadata values
)
print(f"{time:e} s per sample")

# plain data request with Quasi-Dipole Latitude (QDLatitude)
time = get_request_time(
    collection=COLLECTION["A"],
    start_time = "2019-06-01", # both iso-date/time string or datetime.datetime object is accepted
    end_time = "2019-06-02",
    auxiliaries=["QDLat"],
)
print(f"{time:e} s per sample")

# plain data request with Magnetic Local Time (MLT)
time = get_request_time(
    collection=COLLECTION["A"],
    start_time = "2019-06-01", # both iso-date/time string or datetime.datetime object is accepted
    end_time = "2019-06-02",
    auxiliaries=["MLT"],
)
print(f"{time:e} s per sample")


# plain data request with QD-coordinates
time = get_request_time(
    collection=COLLECTION["A"],
    start_time = "2019-06-01", # both iso-date/time string or datetime.datetime object is accepted
    end_time = "2019-06-02",
    auxiliaries=["MLT", "QDLat"],
)
print(f"{time:e} s per sample")


# plain data request with CHAOS-Core model
time = get_request_time(
    collection=COLLECTION["A"],
    start_time = "2019-06-01", # both iso-date/time string or datetime.datetime object is accepted
    end_time = "2019-06-02",
    models=["CHAOS-Core"],
)
print(f"{time:e} s per sample")

# plain data request with CHAOS-Static model
time = get_request_time(
    collection=COLLECTION["A"],
    start_time = "2019-06-01", # both iso-date/time string or datetime.datetime object is accepted
    end_time = "2019-06-02",
    models=["CHAOS-Static"],
)
print(f"{time:e} s per sample")


# plain data request with CHAOS-MMA model (CHAOS-MMA-Primary + CHAOS-MMA-Secondary)
time = get_request_time(
    collection=COLLECTION["A"],
    start_time = "2019-06-01", # both iso-date/time string or datetime.datetime object is accepted
    end_time = "2019-06-02",
    models=["CHAOS-MMA"],
)
print(f"{time:e} s per sample")

# plain data request with CHAOS-MMA-Primary model
time = get_request_time(
    collection=COLLECTION["A"],
    start_time = "2019-06-01", # both iso-date/time string or datetime.datetime object is accepted
    end_time = "2019-06-02",
    models=["CHAOS-MMA-Primary"],
)
print(f"{time:e} s per sample")


# plain data request with CHAOS-MMA-Secondary model
time = get_request_time(
    collection=COLLECTION["A"],
    start_time = "2019-06-01", # both iso-date/time string or datetime.datetime object is accepted
    end_time = "2019-06-02",
    models=["CHAOS-MMA-Secondary"],
)
print(f"{time:e} s per sample")


# plain data request with CHAOS (complete) model (CHAOS-Core + CHAOS-Static + CHAOS-MMA-Primary + CHAOS-MMA-Secondary)
time = get_request_time(
    collection=COLLECTION["A"],
    start_time = "2019-06-01", # both iso-date/time string or datetime.datetime object is accepted
    end_time = "2019-06-02",
    models=["CHAOS"],
)
print(f"{time:e} s per sample")


# plain data request with CHAOS (complete) model (CHAOS-Core + CHAOS-Static + CHAOS-MMA-Primary + CHAOS-MMA-Secondary) / short request
time = get_request_time(
    collection=COLLECTION["A"],
    start_time = "2019-06-01", # both iso-date/time string or datetime.datetime object is accepted
    end_time = "2019-06-01T01:00Z",
    models=["CHAOS"],
)
print(f"{time:e} s per sample")

{'startDate': datetime.datetime(2013, 11, 25, 11, 2, 52), 'stopDate': datetime.datetime(2022, 10, 23, 23, 59, 59)}


Downloading:   0%|          |  [ Elapsed: 00:00, Remaining: ? ] (4.929MB)

86400 ['Spacecraft', 'Latitude', 'B_NEC', 'Radius', 'Longitude']
2.414213e-06 s per sample


Downloading:   0%|          |  [ Elapsed: 00:00, Remaining: ? ] (4.929MB)

86400 ['Spacecraft', 'Latitude', 'B_NEC', 'Radius', 'Longitude']
2.622708e-06 s per sample


...

86400 ['Spacecraft', 'Latitude', 'Radius', 'B_NEC', 'Longitude', 'B_NEC_CHAOS-MMA']
1.453569e-03 s per sample


Downloading:   0%|          |  [ Elapsed: 00:00, Remaining: ? ] (7.003MB)

86400 ['Spacecraft', 'Latitude', 'Radius', 'B_NEC', 'Longitude', 'B_NEC_CHAOS-MMA-Primary']
7.284539e-04 s per sample


Downloading:   0%|          |  [ Elapsed: 00:00, Remaining: ? ] (7.003MB)

86400 ['Spacecraft', 'Latitude', 'Radius', 'B_NEC_CHAOS-MMA-Secondary', 'B_NEC', 'Longitude']
7.103467e-04 s per sample
